## Загрузка датасета

In [1]:
import pandas as pd
import numpy as np

In [25]:
def load_tweets(path):
    df = pd.read_csv(path, sep=';')
    df.columns = [
        'id', 'date', 'mane', 'text', 'type','rep',
        'fav','stcount', 'fol', 'frien', 'listcount', 'dummy'
    ]
    return df

In [26]:
df_pos = load_tweets('data/positive.csv')
df_neg = load_tweets('data/negative.csv')

In [27]:
df_pos.head()

,id,date,mane,text,type,rep,fav,stcount,fol,frien,listcount,dummy
0,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
1,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
2,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
3,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
4,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,0,0,0,40,6,16,0


In [28]:
df_combined = pd.concat([df_neg, df_pos])

## Препроцессинг

In [58]:
from gensim.models import *
from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2

In [63]:
morph = pymorphy2.MorphAnalyzer()
stopwords_list = stopwords.words('russian')

def tokenize_and_preprocess(text):
    words = word_tokenize(text)
    lemmas = [morph.parse(w.lower())[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords_list and w.isalpha()]

In [64]:
from sklearn.model_selection import train_test_split


df_mini, _ = train_test_split(df_combined, train_size=0.05, shuffle=True)

In [65]:
df_mini['text'].head(5)

66356     @seraffimm @AlexeyMazur ну вы как Городские пр...
26821     RT @Fanasenko_L: Ахах разговариваем с чуваком ...
10776     @Matvienko_Irina ну на нас свалили дофига, а в...
100823    собрала ёлку, #tappedout обновился( праздник з...
13756     RT @qadyxeponi: Зима наконец-то пришла в Москв...
Name: text, dtype: object

In [66]:
df_mini['text_prep'] = df_mini['text'].apply(tokenize_and_preprocess)
df_mini['text_prep']

66356     [seraffimm, alexeymazur, городской, придумать,...
26821     [rt, ахи, разговаривать, чувак, питер, вообще,...
10776          [свалить, дофига, лишь, апробация, страдать]
100823    [собрать, ёлка, tappedout, обновиться, праздни...
13756                [rt, qadyxeponi, зима, прийти, москва]
                                ...                        
49648     [добраться, фитнес, думать, отдохнуть, всё, на...
65752     [поругать, сегодня, любимый, уйти, комната, пр...
38404     [lt, присниться, девушка, голубой, волос, это,...
59809     [rt, edabapazni, таки, пройти, мимо, высоко, п...
73040                              [сладкий, сон, lt, http]
Name: text_prep, Length: 11341, dtype: object

In [88]:
dictionary = corpora.Dictionary(df_mini['text_prep'])
dictionary.filter_extremes(no_below = 12, no_above = 0.9, keep_n=None)
dictionary.save('data/tweet.dict')

In [91]:
corpus = [dictionary.doc2bow(text) for text in df_mini['text_prep']]
corpora.MmCorpus.serialize('data/tweet.model', corpus)

In [92]:
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(9, 1)],
 [(10, 1), (11, 1), (12, 1), (13, 1)],
 [(3, 1), (14, 1), (15, 1), (16, 1)],
 [(17, 1)],
 [(18, 1), (19, 1), (20, 1), (21, 1), (22, 1)],
 [(18, 1)],
 [(3, 1), (23, 1), (24, 1)],
 [(19, 1), (25, 1)],
 [(26, 2), (27, 1), (28, 1), (29, 1), (30, 1)],
 [(31, 1)],
 [(26, 1), (30, 1), (32, 1), (33, 1)],
 [(3, 1),
  (18, 1),
  (34, 2),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1)],
 [(30, 1), (43, 1), (44, 1), (45, 1), (46, 1)],
 [(33, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1)],
 [(30, 1), (43, 1), (53, 1), (54, 1), (55, 1), (56, 1)],
 [(31, 1), (57, 1)],
 [(0, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1)],
 [(9, 1), (64, 1), (65, 1)],
 [(66, 1), (67, 1)],
 [],
 [(68, 1), (69, 1)],
 [(70, 1), (71, 1)],
 [(72, 1), (73, 1)],
 [(26, 1), (74, 1), (75, 1), (76, 1)],
 [(77, 1), (78, 1), (79, 1)],
 [(28, 1), (80, 1)],
 [(31, 1), (58, 1)],
 [(33,

## Моделирование тем

In [112]:
NUM_TOPICS = 5

In [113]:
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=NUM_TOPICS, chunksize=64, update_every=1, passes=2)

In [114]:
lda.show_topics(num_topics=NUM_TOPICS, num_words=5, formatted=True)

[(0,
  '0.061*"ещё" + 0.028*"вообще" + 0.028*"говорить" + 0.024*"всё" + 0.022*"писать"'),
 (1, '0.139*"http" + 0.126*"rt" + 0.065*"d" + 0.043*"хотеть" + 0.025*"всё"'),
 (2,
  '0.074*"это" + 0.033*"год" + 0.029*"новый" + 0.024*"сидеть" + 0.021*"делать"'),
 (3,
  '0.066*"день" + 0.054*"весь" + 0.047*"очень" + 0.046*"всё" + 0.035*"хороший"'),
 (4,
  '0.059*"сегодня" + 0.049*"rt" + 0.037*"человек" + 0.035*"просто" + 0.029*"блин"')]

## Визуализация

In [115]:
import pyLDAvis as ldavis
import pyLDAvis.gensim_models as gensim_models
import warnings

In [116]:
warnings.filterwarnings('ignore')
vis_data = gensim_models.prepare(lda, corpus, dictionary)
ldavis.display(vis_data)

## Интерпретация тематик

In [117]:
lda.show_topics(num_topics=NUM_TOPICS, num_words=50, formatted=True)

[(0,
  '0.061*"ещё" + 0.028*"вообще" + 0.028*"говорить" + 0.024*"всё" + 0.022*"писать" + 0.021*"это" + 0.019*"смотреть" + 0.018*"слава" + 0.018*"понять" + 0.017*"школа" + 0.017*"звонить" + 0.016*"казаться" + 0.016*"наш" + 0.015*"друг" + 0.013*"приходить" + 0.012*"таки" + 0.012*"курс" + 0.012*"дело" + 0.011*"плохо" + 0.010*"маленький" + 0.010*"помнить" + 0.010*"домой" + 0.010*"понравиться" + 0.010*"гора" + 0.009*"снова" + 0.009*"улица" + 0.009*"приехать" + 0.009*"новогодний" + 0.009*"телефон" + 0.008*"получить" + 0.008*"закончиться" + 0.008*"красивый" + 0.007*"отлично" + 0.007*"холодно" + 0.007*"бог" + 0.007*"х" + 0.007*"фото" + 0.007*"душа" + 0.007*"работа" + 0.007*"сессия" + 0.007*"никакой" + 0.007*"каждый" + 0.007*"слово" + 0.006*"праздник" + 0.006*"второй" + 0.006*"нужный" + 0.006*"тренировка" + 0.006*"гулять" + 0.006*"видео" + 0.006*"суббота"'),
 (1,
  '0.139*"http" + 0.126*"rt" + 0.065*"d" + 0.043*"хотеть" + 0.025*"всё" + 0.025*"мочь" + 0.021*"любить" + 0.019*"спасибо" + 0.019*"лю

Тематика под номером 2 (если считать от 0) говорит о праздновании нового года

### Метрики

In [120]:
lda.log_perplexity(corpus)

-6.548832482630697

In [124]:
coherence_model_lda = CoherenceModel(
    model=lda,
    texts=df_mini['text_prep'],
    dictionary=dictionary,
    coherence='c_v')
coherence_model_lda.get_coherence_per_topic()

[0.3752865478785906,
 0.15966101828979531,
 0.4647082593675795,
 0.24983513796716453,
 0.21338513802896095]